In [1]:
%run ../__init__.py

## Data preprocessing with Tfidf

In [2]:
budget_df = pd.read_csv('../data/budget_df_cleaned1.csv', index_col=0)

In [18]:
budget_df.head()

,Function,Use,Sharing,Reporting,Student_Type,Position_Type,Object_Type,Pre_K,Operating_Status,Object_Description,...,Location_Description,FTE,Function_Description,Facility_or_Department,Position_Extra,Total,Program_Description,Fund_Description,Text_1,combined_text
0,Function__Teacher Compensation,Use__Instruction,Sharing__School Reported,Reporting__School,Student_Type__NO_LABEL,Position_Type__Teacher,Object_Type__NO_LABEL,Pre_K__NO_LABEL,Operating_Status__PreK-12 Operating,,...,,1.000000,,,kindergarten,50471.810,kindergarten,general fund,,teacher elementary kindergar...
1,Function__NO_LABEL,Use__NO_LABEL,Sharing__NO_LABEL,Reporting__NO_LABEL,Student_Type__NO_LABEL,Position_Type__NO_LABEL,Object_Type__NO_LABEL,Pre_K__NO_LABEL,Operating_Status__Non-Operating,contractor service,...,,0.420663,rgn gob,,undesignated,3477.860,building improvement service,,building improvement service,contractor service bond expenditure building f...
2,Function__Teacher Compensation,Use__Instruction,Sharing__School Reported,Reporting__School,Student_Type__Unspecified,Position_Type__Teacher,Object_Type__Base Salary/Compensation,Pre_K__Non PreK,Operating_Status__PreK-12 Operating,personal service teacher,...,,1.000000,,,teacher,62237.130,instruction regular,general purpose school,,personal service teacher tcher 2nd grade ...
3,Function__Substitute Compensation,Use__Instruction,Sharing__School Reported,Reporting__School,Student_Type__Unspecified,Position_Type__Substitute,Object_Type__Benefits,Pre_K__NO_LABEL,Operating_Status__PreK-12 Operating,employee benefit,...,,0.420663,unalloc budget school,,professional instructional,22.300,general middle junior high sch,,regular instruction,employee benefit teacher sub general fund teac...
4,Function__Substitute Compensation,Use__Instruction,Sharing__School Reported,Reporting__School,Student_Type__Unspecified,Position_Type__Teacher,Object_Type__Substitute Compensation,Pre_K__NO_LABEL,Operating_Status__PreK-12 Operating,teacher coverage teacher,...,,0.420663,non project,,professional instructional,54.166,general high school education,,regular instruction,teacher coverage teacher teacher sub general f...


In [4]:
for col in budget_df[word_predictors]:
    budget_df[col] = budget_df[word_predictors].loc[:,[col]].fillna(' ')

In [5]:
def create_tfidf(word_column):
    tfidf = TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=100)
    col_tfidf = tfidf.fit_transform(budget_df[word_column])
    
    return col_tfidf

In [6]:
SubFund_Description_tfidf = create_tfidf('SubFund_Description')
Job_Title_Description_tfidf = create_tfidf('Job_Title_Description')
Text_3_tfidf = create_tfidf('Text_3')
Text_4_tfidf= create_tfidf('Text_4')
Sub_Object_Description_tfidf= create_tfidf('Sub_Object_Description')
Location_Description_tfidf= create_tfidf('Location_Description')
Function_Description_tfidf= create_tfidf('Function_Description')
Facility_or_Department_tfidf= create_tfidf('Facility_or_Department')
Position_Extra_tfidf= create_tfidf('Position_Extra')
Program_Description_tfidf= create_tfidf('Program_Description')
Fund_Description_tfidf= create_tfidf('Fund_Description')
Text_1_tfidf= create_tfidf('Text_1')
Text_2_tfidf= create_tfidf('Text_2')
Object_Description = create_tfidf('Object_Description')

In [7]:
# combined_text_tfidf= create_tfidf('combined_text')

# combined_text_tfidf.get_shape()

In [9]:
tfidf_list = [SubFund_Description_tfidf, Job_Title_Description_tfidf, Text_3_tfidf, Text_4_tfidf,\
                      Sub_Object_Description_tfidf, Location_Description_tfidf, Function_Description_tfidf,\
                       Facility_or_Department_tfidf, Position_Extra_tfidf, Program_Description_tfidf,\
                      Fund_Description_tfidf, Text_1_tfidf, Text_2_tfidf, Object_Description]

In [10]:
from scipy import sparse
from scipy.sparse import hstack

budget_tfidf = hstack([SubFund_Description_tfidf, Job_Title_Description_tfidf, Text_3_tfidf, Text_4_tfidf,\
                      Sub_Object_Description_tfidf, Location_Description_tfidf, Function_Description_tfidf,\
                       Facility_or_Department_tfidf, Position_Extra_tfidf, Program_Description_tfidf,\
                      Fund_Description_tfidf, Text_1_tfidf, Text_2_tfidf, Object_Description])

In [11]:
budget_tfidf.get_shape()

(400277, 4717)

In [12]:
targets

['Function',
 'Use',
 'Sharing',
 'Reporting',
 'Student_Type',
 'Position_Type',
 'Object_Type',
 'Pre_K',
 'Operating_Status']

## Model Prediction for Operating_status

In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [14]:
def nlp_pipe(X, y, col):
     
    X_train, X_val, y_train, y_val = train_test_split(X, 
                                                    y, 
                                                    test_size=.2, 
                                                    random_state=42)
    
    nlp_pipe = Pipeline([
    ('clf', LogisticRegression(random_state=42, multi_class='ovr'))])

    
    model = nlp_pipe.fit(X_train, y_train)
    
    results = {'target': col,
               'train_score': nlp_pipe.score(X_train, y_train),
               'test_score': nlp_pipe.score(X_val, y_val)}
    
    return model, results

In [ ]:
op_status = nlp_pipe(Text_1_tfidf, le.fit_transform(budget_df['Operating_Status']), 'Operating Status')

In [17]:
model_results = []
for predictor in word_predictors:  
    y = le.fit_transform(budget_df[predictor])
    print(predictor)
for item in tfidf_list:
    X = item
for col in targets:
    col = col
    print()
    results = nlp_pipe(X, y, col)
    model_results.append(results)

Object_Description
Text_2
SubFund_Description
Job_Title_Description
Text_3
Text_4
Sub_Object_Description
Location_Description
Function_Description
Facility_or_Department
Position_Extra
Program_Description
Fund_Description
Text_1



KeyboardInterrupt: 

In [40]:
op_status = nlp_pipe(Text_1_tfidf, le.fit_transform(budget_df['Operating_Status']), 'Operating Status')

In [ ]:
op_status

## Model with combined sparse matrix

In [ ]:
# le = LabelEncoder()
# budget_df['Operating_status_num'] = le.fit_transform(budget_df['Operating_Status'])

In [ ]:
X = budget_tfidf
y = le.fit_transform(budget_df['Operating_Status'])
X_train, X_val, y_train, y_val = train_test_split(X,y,random_state = 42)

In [21]:
logreg.score(X_train,y_train)

0.98564657053299887

In [ ]:
logreg.score(X_test,y_test)

In [23]:
y_predicted = logreg.predict(X_test)

In [24]:
y_predicted[:50]

array([2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2,
       0, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 0, 2, 0, 2, 2, 2, 2, 2, 2, 2,
       2, 0, 2, 2])

In [25]:
list(le.classes_)
list(le.inverse_transform([2, 0, 1]))

['Operating_Status__PreK-12 Operating',
 'Operating_Status__Non-Operating',
 'Operating_Status__Operating, Not PreK-12']

## Figure out how to combine model of categorical columns and numerical

### running into memory issue for get_dummies for the 9 cat

In [ ]:
pl = Pipeline([
        ('union', FeatureUnion(
            transformer_list = [
                ('numeric_features', Pipeline([
                    ('selector', get_numeric_data),
                    ('imputer', Imputer())
                ])),
                ('text_features', Pipeline([
                    ('selector', get_text_data),
                    ('vectorizer', HashingVectorizer(token_pattern=TOKENS_ALPHANUMERIC,
                                                     non_negative=True, norm=None, binary=False,
                                                     ngram_range=(1, 2))),
                    ('dim_red', SelectKBest(chi2, chi_k))
                ]))
             ]
        )),
        ('int', SparseInteractions(degree=2)),
        ('scale', MaxAbsScaler()),
        ('clf', OneVsRestClassifier(LogisticRegression()))
    ])
